In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import spacy
import edsnlp
import edsnlp.connectors as c
import pandas

In [ ]:
import spacy

In [ ]:
import os
os.getcwd()

In [ ]:
###################################################################
############## UPDATE THE ATTRIBUTES THAT CONTAINS NOTHING ########
###################################################################

import os

directory = "NLP_diabeto_annotated/clean"  # Replace with the actual directory path
n = 0
for filename in os.listdir(directory):
    if filename.endswith('.ann'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r+') as file:
            lines = file.readlines()
            file.seek(0)
            for line in lines:
                if line.startswith('A') and any(keyword in line for keyword in ['Negation']):
                    line = line.rstrip() + ' neg\n'
                    n+=1
                elif line.startswith('A') and any(keyword in line for keyword in ['Allergie']):
                    line = line.rstrip() + ' is_allergie\n'
                    n+=1
                elif line.startswith('A') and any(keyword in line for keyword in ['Family']):
                    line = line.rstrip() + ' is_family\n'
                    n+=1
                    
                file.write(line)
            file.truncate()
print(n,' Attribute updated')

In [ ]:
import os 
os.getcwd()

In [ ]:
####################################
# TURN THE ATRIBUTES TECH INTO ENT #
####################################

import re
import os

PATH = 'NLP_diabeto_annotated/clean'
pattern = 'T(\d+)'
n = 0
for filename in os.listdir(PATH):
    if filename.endswith(".ann"):
        # Read in the file contents
        with open(os.path.join(PATH, filename), 'r') as f:
            lines = f.readlines()
        to_del = []
        for i in range(1,len(lines)):
            current_line = lines[i]
            previous_line = lines[i-1]

            #if current line start with A and contain the word "Tech"
            if current_line.startswith('A') and 'Tech' in current_line:
                if previous_line.startswith('T'):
                    ent_id = re.findall(pattern, previous_line)[0]
                    if ent_id in current_line:
                        #the Chemical_and_drugs string is in the previous line is replaced by the last word of the current line
                        lines[i-1] = lines[i-1].replace('Chemical_and_drugs', current_line.split()[-1])
                        #the current line is deleted
                        to_del.append(i)
                        n += 1

        # Delete the lines
        for i in sorted(to_del, reverse=True):
            del lines[i]
        # Write the new contents to the file
        with open(os.path.join(PATH, filename), 'w') as f:
            f.writelines(lines) 
                        
print(n,' Attribute updated')
                        

In [ ]:
#INPUT PATH
PATH = 'NLP_diabeto_annotated/clean'

#OUTPUT PATH
PATH_train = 'NLP_diabeto_annotated/train'
PATH_test = 'NLP_diabeto_annotated/test'
PATH_val = 'NLP_diabeto_annotated/val'

treshold = 0.8

brat = c.BratConnector(PATH, attributes = {"Action":"action",'Certainty':'certainty','Temporality':'temporality',
                                             'Negation':'negation','Allergie':'allergie','Family':'family'})
empty = spacy.blank("fr")
df = brat.brat2docs(empty)

# TRAIN = df[:int(len(df)*0.8)]
# TEST = df[int(len(df)*0.8): int(len(df)*0.9)]
# VAL = df[int(len(df)*0.9):]


import numpy as np
import random
from collections import Counter

draw = 100
attributes = ['action', 'certainty', 'temporality', 'negation', 'allergie', 'family']
n = len(df)

#total
counter_tot = Counter(attr for doc in df for ent in doc.ents for attr in attributes if getattr(ent._,attr)) #Full distribution of the attributes

# Index Sampling and counter calculation
index_list = [random.sample(range(n), int(0.2*n)) for _ in range(draw)]
half = int(0.5 * len(index_list[0]))
test_indices = [indices[:half] for indices in index_list]
val_indices = [indices[half:] for indices in index_list]
train_indices = [list(set(range(n)) - set(indices)) for indices in index_list]

counter = {attr: np.zeros(draw, dtype=int) for attr in attributes}
for d, indices in enumerate(test_indices):
    test = [df[i] for i in indices]
    for doc in test:
        for ent in doc.ents:
            for attr in attributes:
                if getattr(ent._, attr):
                    counter[attr][d] += 1

# Normalization
for attr in attributes:
    counter[attr] = np.round(counter[attr] / counter_tot[attr], 2)

# Best Index
totals = np.array([sum(counter[attr][d] for attr in attributes) for d in range(draw)])
best_index = np.argmin(np.abs(0.2 * len(attributes) - totals)**2)


#Dataset generation
TEST = [df[i] for i in index_list[best_index][:int(0.5*len(index_list[best_index]))]]
VAL = [df[i] for i in index_list[best_index][int(0.5*len(index_list[best_index])):]]
TRAIN = [df[i] for i in range(n) if i not in index_list[best_index]]


print('Train size : ', len(TRAIN))
print('Test size : ', len(TEST))
print('Val size : ', len(VAL))

brat = c.BratConnector(PATH_train,attributes = {"Action":"action",'Certainty':'certainty','Temporality':'temporality',
                                             'Negation':'negation','Allergie':'allergie','Family':'family'})
brat.docs2brat(TRAIN)

print('Train saved')

brat = c.BratConnector(PATH_test,attributes = {"Action":"action",'Certainty':'certainty','Temporality':'temporality',
                                             'Negation':'negation','Allergie':'allergie','Family':'family'})
brat.docs2brat(TEST)

print('Test saved')


brat = c.BratConnector(PATH_val,attributes = {"Action":"action",'Certainty':'certainty','Temporality':'temporality',
                                             'Negation':'negation','Allergie':'allergie','Family':'family'})
brat.docs2brat(VAL)

print('Val saved')

In [ ]:
from collections import Counter
labels = []
attributes = ['action', 'certainty', 'temporality', 'negation', 'allergie', 'family']
for doc in TEST:
    for ent in doc.ents:
        labels.append(ent.label_)
        for attr in attributes:
            if getattr(ent._,attr):
                labels.append(attr)
#we print the labels and their frequency
Counter(labels)

In [ ]:
PATH = 'attr/full'

#OUTPUT PATH
PATH_train = 'attr/train'
PATH_test = 'attr/test'
PATH_val = 'attr/val'

treshold = 0.8

brat = c.BratConnector(PATH, attributes = {"Action":"action",'Certainty':'certainty','Temporality':'temporality','Negation':'negation','Allergie':'allergie','Family':'family'})
empty = spacy.blank("fr")
df = brat.brat2docs(empty)

TRAIN = df[:int(len(df)*0.8)]
TEST = df[int(len(df)*0.8): int(len(df)*0.9)]
VAL = df[int(len(df)*0.9):]

In [ ]:
from collections import Counter
import numpy
import random



draw = 500


attributes = ['action', 'certainty', 'temporality', 'negation', 'allergie', 'family']

counter_tot = {attr:0 for attr in attributes}

#total
for doc in df:
    for ent in doc.ents:
        for attr in attributes:
            if getattr(ent._,attr):
                 counter_tot[attr] += 1

n = len(df)

index_list = []
counter = {attr:[0 for _ in range(draw)] for attr in attributes}

for d in range(draw):

    index = random.sample(range(n),int(0.2*n))
    index_list.append(index)

    test = [df[i] for i in index[:int(0.5*len(index))]]
    val = [df[i] for i in index[int(0.5*len(index)):]]
    train = [df[i] for i in range(n) if i not in index]
    

    for doc in test:
        for ent in doc.ents:
            for attr in attributes:
                if getattr(ent._,attr):
                     counter[attr][d] += 1




for attr in attributes:
    for d in range(draw):
        counter[attr][d] = round(counter[attr][d] / counter_tot[attr],2)

best = 0.0
best_index = 0
for d in range(draw):
    tot = 0.0
    for attr in attributes:
        tot+=counter[attr][d]
    if (0.2*len(attributes) - tot)**2 < (0.2*len(attributes) - best)**2:
        best = tot
        best_index = d

index_list[best_index]

In [ ]:
import numpy as np
import random
from collections import Counter

draw = 10
attributes = ['action', 'certainty', 'temporality', 'negation', 'allergie', 'family']
n = len(df)

#total
counter_tot = Counter(attr for doc in df for ent in doc.ents for attr in attributes if getattr(ent._,attr)) #Full distribution of the attributes

# Index Sampling and counter calculation
index_list = [random.sample(range(n), int(0.2*n)) for _ in range(draw)]
half = int(0.5 * len(index_list[0]))
test_indices = [indices[:half] for indices in index_list]
val_indices = [indices[half:] for indices in index_list]
train_indices = [list(set(range(n)) - set(indices)) for indices in index_list]

counter = {attr: np.zeros(draw, dtype=int) for attr in attributes}
for d, indices in enumerate(test_indices):
    test = [df[i] for i in indices]
    for doc in test:
        for ent in doc.ents:
            for attr in attributes:
                if getattr(ent._, attr):
                    counter[attr][d] += 1

# Normalization
for attr in attributes:
    counter[attr] = np.round(counter[attr] / counter_tot[attr], 2)

# Best Index
totals = np.array([sum(counter[attr][d] for attr in attributes) for d in range(draw)])
best_index = np.argmin(np.abs(0.2 * len(attributes) - totals)**2)

In [ ]:
best_index

In [ ]:

PATH_train = 'attr/train'
PATH_test = 'attr/test'
PATH_val = 'attr/val'

treshold = 0.8

brat = c.BratConnector(PATH_test, attributes = {"Action":"action",'Certainty':'certainty','Temporality':'temporality','Negation':'negation','Allergie':'allergie','Family':'family'})
empty = spacy.blank("fr")
TEST = brat.brat2docs(empty)

attributes = ['action', 'certainty', 'temporality', 'negation', 'allergie', 'family']

counter_tot = {attr:0 for attr in attributes}

a = Counter()
for doc in TEST:
    for ent in doc.ents:
        for attr in attributes:
            if getattr(ent._,attr):
                
                 a[getattr(ent._,attr)] += 1

a

In [ ]:
#INPUT PATH
PATH = 'attr/full'

#OUTPUT PATH
PATH_train = 'attr/train'
PATH_test = 'attr/test'
PATH_val = 'attr/val'

treshold = 0.8

brat = c.BratConnector(PATH_test, attributes = {"Action":"action",'Certainty':'certainty','Temporality':'temporality',
                                             'Negation':'negation','Allergie':'allergie','Family':'family'})
empty = spacy.blank("fr")
df = brat.brat2docs(empty)



for doc in df:
    for ent in doc:
        if ent._.action:
            print('a')
            print(ent,'--', ent._.action)

In [ ]:
brat = c.BratConnector('testtest/', attributes = {"action":"action",'certainty':'certainty','temporality':'temporality','negation':'negation','allergie':'allergie','family':'family'})

brat.docs2brat(df)

In [ ]:
for doc in df:
    for ent in doc.ents:
        if ent._.action:
            print('ok')